# Julia notebook for the analytics of the multilevel selection Wright-Fisher process

**First, we will start with some preliminaries about matrix exponentiation and logarithms in order to bring the Wright-Fisher to a form where the Campbell-Baker-Hausdorf formula can be applied**

In [14]:
#Required packages
using Distributions
using Combinatorics
using Iterators
using ClusterManagers
using DataFrames
using Plots

#Function to generate the state space
function states(state_vec)
	state_zero = collect(permutations(state_vec[1]))
	for i=1:length(state_vec)
		additional=collect(permutations(state_vec[i]))
		state_zero=append!(state_zero,additional)
	end
	state_zero=unique(state_zero)
	return state_zero
end

function multinomial2(k)
    s = 0
    result = 1
    @inbounds for i in k
        s += i
        result *= binomial(BigInt(s), i)
    end
    result
end
#CBH formula to third order

function commute_mat(A,B)
    com=A*B-B*A
    return com
end

function CBH(A,B)
    Z=A+B+1/2*commute_mat(A,B)+1/12*(commute_mat(A,commute_mat(A,B))+commute_mat(B,commute_mat(B,A)))
    return Z
end

#Lets start defining a fitness function, in the packge model literature is costumary to make use of the geometric average.
function gmean(A::Array{Int64,1})
	# geometric average of a vector
	#value=prod(A)^(1/length(A))
    value=A[1]/sum(A)*(1-A[1]/sum(A))
	return value
    end;
    #value of the transition matrix
function t_ij(u,v)
    #probability from state u to state v
    p=deepcopy(u)/sum(u)
    prob=multinomial2(v)*prod(p.^v)
    prob=convert(Float64,prob)
    return prob
end


function T_construct(N::Int64,seqtypes::Int64)
    state_vec=collect(partitions(N+seqtypes,seqtypes))-1
    state_zero=states(state_vec)
    state_zero=sort(state_zero, lt=(x,y)->isless(x[1], y[1]))
    T=[t_ij(state_zero[i],state_zero[j]) for i in 1:length(state_zero), j in 1:length(state_zero)]
    return T
end
    

T_construct (generic function with 1 method)

**CBH example**

In [ ]:
#For two matrices
A=[0.3 0;0 0.2]
B=[0.6 0.5;0.4 0.5]

In [ ]:
#Make sure they do not commute
commute_mat(A,B)

In [ ]:
#Calculate the CBH matrix 
Z=CBH(logm(A),logm(B))

In [ ]:
eigvecs(A*B)

In [ ]:
eigvecs(Z)

In [ ]:
#Study the eigenvectors
eigvecs(*(A,B))

In [ ]:
eigvecs(expm(B))

# logTij equivalence to the differential operator in Kimura

In [70]:
#Parameters
N=150;
seqtypes=2;

The state space of the protocell model is are the solutions to the Diophantine equations x+y=N where N is the ploidy. Here is a way of generating it.

In [71]:
state_vec=collect(partitions(N+seqtypes,seqtypes))-1
state_zero=states(state_vec)
state_zero=sort(state_zero, lt=(x,y)->isless(x[1], y[1]))

151-element Array{Array{Int64,1},1}:
 [0, 150] 
 [1, 149] 
 [2, 148] 
 [3, 147] 
 [4, 146] 
 [5, 145] 
 [6, 144] 
 [7, 143] 
 [8, 142] 
 [9, 141] 
 [10, 140]
 [11, 139]
 [12, 138]
 ⋮        
 [139, 11]
 [140, 10]
 [141, 9] 
 [142, 8] 
 [143, 7] 
 [144, 6] 
 [145, 5] 
 [146, 4] 
 [147, 3] 
 [148, 2] 
 [149, 1] 
 [150, 0] 

In [72]:
w=[gmean(state_zero[i]) for i in 1:length(state_zero)]
W=Diagonal(w)

151×151 Diagonal{Float64}:
 0.0   ⋅           ⋅          ⋅      …   ⋅       ⋅          ⋅           ⋅ 
  ⋅   0.00662222   ⋅          ⋅          ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅          0.0131556   ⋅          ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅         0.0196      ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅          ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅      …   ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅          ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅          ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅          ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅          ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅      …   ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅          ⋅       ⋅          ⋅           ⋅ 
  ⋅    ⋅           ⋅          ⋅          ⋅       ⋅          ⋅           ⋅

In [73]:
T_construct(N,seqtypes)

151×151 Array{Float64,2}:
 1.0           0.0           0.0           …  0.0           0.0         
 0.36665       0.36911       0.184555         0.0           0.0         
 0.133527      0.270662      0.272491         6.1113e-278   5.50567e-282
 0.048296      0.147845      0.224785         1.04903e-251  1.42725e-255
 0.0173476     0.0712915     0.145513         4.30223e-233  7.85796e-237
 0.00618745    0.032004      0.0822173     …  1.17571e-218  2.70279e-222
 0.00219121    0.0136951     0.0425118        7.33333e-207  2.03704e-210
 0.000770401   0.00565679    0.0206295        6.87617e-197  2.24397e-200
 0.000268883   0.00227225    0.00953704       2.98606e-188  1.12153e-191
 9.3149e-5     0.000891852   0.00424104       1.24095e-180  5.28065e-184
 3.2027e-5     0.000343147   0.00182603    …  8.10085e-174  3.85755e-177
 1.09278e-5    0.000129718   0.000764776      1.18284e-167  6.24042e-171
 3.69976e-6    4.82578e-5    0.000312627      5.01519e-162  2.90735e-165
 ⋮                       

In [74]:
T=[t_ij(state_zero[i],state_zero[j]) for i in 1:length(state_zero), j in 1:length(state_zero)]

151×151 Array{Float64,2}:
 1.0           0.0           0.0           …  0.0           0.0         
 0.36665       0.36911       0.184555         0.0           0.0         
 0.133527      0.270662      0.272491         6.1113e-278   5.50567e-282
 0.048296      0.147845      0.224785         1.04903e-251  1.42725e-255
 0.0173476     0.0712915     0.145513         4.30223e-233  7.85796e-237
 0.00618745    0.032004      0.0822173     …  1.17571e-218  2.70279e-222
 0.00219121    0.0136951     0.0425118        7.33333e-207  2.03704e-210
 0.000770401   0.00565679    0.0206295        6.87617e-197  2.24397e-200
 0.000268883   0.00227225    0.00953704       2.98606e-188  1.12153e-191
 9.3149e-5     0.000891852   0.00424104       1.24095e-180  5.28065e-184
 3.2027e-5     0.000343147   0.00182603    …  8.10085e-174  3.85755e-177
 1.09278e-5    0.000129718   0.000764776      1.18284e-167  6.24042e-171
 3.69976e-6    4.82578e-5    0.000312627      5.01519e-162  2.90735e-165
 ⋮                       

In [91]:
Z=real(logm(T))

151×151 Array{Float64,2}:
  0.0           0.0           0.0          …   0.0           0.0        
  0.608535     -1.36299       0.945946         7.93941e-19   1.50848e-16
 -0.00210468    1.21496      -2.7131          -5.1173e-16   -1.70552e-16
  1.8697e-5    -0.00634044    1.81923          8.37537e-17  -1.86038e-16
 -2.79153e-7    7.55577e-5   -0.012734        -2.53616e-15   1.63407e-17
  5.94524e-9   -1.41883e-6    0.000190845  …   1.61354e-14  -7.06741e-16
 -1.657e-10     3.64853e-8   -4.32707e-6      -5.29999e-14   1.42277e-15
  5.67351e-12  -1.19129e-9    1.30573e-7       1.39669e-13  -3.62584e-15
 -1.11962e-13   4.19479e-11  -4.79282e-9      -3.13807e-13   8.14995e-15
 -2.131e-13     8.07643e-12  -1.32842e-11      6.08968e-13  -1.477e-14  
  3.62017e-13  -1.71303e-11   3.93118e-10  …  -1.06821e-12   2.37072e-14
 -5.33665e-13   2.618e-11    -6.29984e-10      1.73149e-12  -3.50806e-14
  7.10057e-13  -3.57799e-11   8.80759e-10     -2.56202e-12   5.04362e-14
  ⋮                      

In [92]:
Tridiagonal(Z)

151×151 Tridiagonal{Float64}:
 0.0        0.0        ⋅          ⋅       …    ⋅          ⋅        ⋅      
 0.608535  -1.36299   0.945946    ⋅            ⋅          ⋅        ⋅      
  ⋅         1.21496  -2.7131     1.8802        ⋅          ⋅        ⋅      
  ⋅          ⋅        1.81923   -4.05025       ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅         2.42133       ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅          ⋅       …    ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅          ⋅            ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅          ⋅            ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅          ⋅            ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅          ⋅            ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅          ⋅       …    ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅          ⋅            ⋅          ⋅        ⋅      
  ⋅          ⋅         ⋅          ⋅            ⋅          ⋅        ⋅  

In [84]:
diagind(Z)
mean(Z[diagind(Z)])
Z[diagind(Z)]

151-element Array{Float64,1}:
   0.0    
  -1.36299
  -2.7131 
  -4.05025
  -5.37434
  -6.68531
  -7.98305
  -9.26748
 -10.5385 
 -11.7958 
 -13.0379 
 -14.2584 
 -15.4363 
   ⋮      
 -14.2678 
 -13.0397 
 -11.796  
 -10.5385 
  -9.26748
  -7.98305
  -6.68531
  -5.37434
  -4.05025
  -2.7131 
  -1.36299
   0.0    

In [80]:
d=-2*ones(length(state_zero))
dl=ones(length(state_zero)-1)
du=ones(length(state_zero)-1)
D=Tridiagonal(dl, d, du)

151×151 Tridiagonal{Float64}:
 -2.0   1.0    ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  1.0  -2.0   1.0    ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅    1.0  -2.0   1.0    ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅    1.0  -2.0   1.0    ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅    1.0  -2.0   1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅    1.0  -2.0  …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅    1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅   

In [83]:
kimura=D*W
kimura[diagind(kimura)]

151-element Array{Float64,1}:
 -0.0      
 -0.0132444
 -0.0263111
 -0.0392   
 -0.0519111
 -0.0644444
 -0.0768   
 -0.0889778
 -0.100978 
 -0.1128   
 -0.124444 
 -0.135911 
 -0.1472   
  ⋮        
 -0.135911 
 -0.124444 
 -0.1128   
 -0.100978 
 -0.0889778
 -0.0768   
 -0.0644444
 -0.0519111
 -0.0392   
 -0.0263111
 -0.0132444
 -0.0      

In [90]:
D*W

151×151 Array{Float64,2}:
 -0.0   0.00662222   0.0         0.0     …   0.0         0.0          0.0
  0.0  -0.0132444    0.0131556   0.0         0.0         0.0          0.0
  0.0   0.00662222  -0.0263111   0.0196      0.0         0.0          0.0
  0.0   0.0          0.0131556  -0.0392      0.0         0.0          0.0
  0.0   0.0          0.0         0.0196      0.0         0.0          0.0
  0.0   0.0          0.0         0.0     …   0.0         0.0          0.0
  0.0   0.0          0.0         0.0         0.0         0.0          0.0
  0.0   0.0          0.0         0.0         0.0         0.0          0.0
  0.0   0.0          0.0         0.0         0.0         0.0          0.0
  0.0   0.0          0.0         0.0         0.0         0.0          0.0
  0.0   0.0          0.0         0.0     …   0.0         0.0          0.0
  0.0   0.0          0.0         0.0         0.0         0.0          0.0
  0.0   0.0          0.0         0.0         0.0         0.0          0.0
  ⋮         

# Dependence with N

In [13]:
ploidy=vcat(collect(5:10),5*collect(3:20));
n_dep=DataFrame();
n_dep[:ploidy]=ploidy;
trace=Array(Float64,length(ploidy));
seqtypes=2;

In [8]:
for x in ploidy   
    T=T_construct(x,seqtypes)
    Z=real(logm(T))
    

    
    
end

,ploidy
1,5
2,6
3,7
4,8
5,9
6,10
7,15
8,20
9,25
10,30
